<a href="https://colab.research.google.com/github/pratikiiitb2013/NLP_END_P1/blob/main/Session4/END_assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext import data
from torchtext import datasets

import spacy
nlp = spacy.load('en')

SEED = 1

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:08<00:00, 9.57MB/s]


In [3]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data.examples[0]))

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000
{'text': ['I', 'would', 'like', 'to', 'know', 'why', 'John', 'Amos', 'left', 'the', 'show', ',', 'and', 'how', 'did', 'he', 'die', 'off', 'the', 'show', 'again', '?', 'I', 'could', "n't", 'relate', 'to', 'everything', ',', 'but', 'sometimes', 'they', 'hit', 'home', 'with', 'the', 'problems', 'they', 'were', 'facing', '.', 'By', 'the', 'way', ',', 'did', 'they', 'ever', 'make', 'it', 'out', 'of', 'the', 'ghetto', '?', 'I', 'think', 'the', 'episode', 'with', 'the', 'black', 'Jesus', 'was', 'my', 'favorite', '.', 'We', 'got', 'to', 'see', 'them', 'experience', 'a', 'few', 'good', 'times', '.', 'something', 'they', 'did', "n't", 'have', 'very', 'often', '.', 'I', 'wish', 'they', 'would', 'bring', 'the', 'show', 'back', '.', 'During', 'the', 'daytime', 'so', 'people', 'can', 'actually', 'stay', 'up', 'to', 'watch', '.', 'I', 'do', "n't", 'think', 'a', 'movie', 'or', 'a', 'new', 'show', '

In [5]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
 99%|█████████▉| 397794/400000 [00:17<00:00, 20293.61it/s]

In [6]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [7]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [8]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        self.rnns = nn.ModuleList()
        for i in range(3):
            input_size = embedding_dim if i == 0 else hidden_dim
            self.rnns.append(nn.LSTM(input_size, hidden_dim, num_layers=1,bidirectional=False,dropout=dropout))

        self.fc = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        text_lengths = text_lengths.cpu()
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        o = packed_embedded
        for rnn in self.rnns:
            packed_output, (o, cell) = rnn(o)
        hidden = o
        hidden = hidden.squeeze(0)

            
        return self.fc(hidden)

In [9]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [10]:
###### checking to see if 3 LSTM layers are there ##########
print(model)

RNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (rnns): ModuleList(
    (0): LSTM(100, 256, dropout=0.2)
    (1): LSTM(256, 256, dropout=0.2)
    (2): LSTM(256, 256, dropout=0.2)
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,919,721 trainable parameters


In [12]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [13]:

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-1.5256, -0.7502, -0.6540,  ...,  0.3434, -1.0703, -0.8743],
        [ 0.2053,  0.3051,  0.5357,  ..., -0.4203,  0.2828,  0.3642],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3598, -0.2233, -0.4987,  ..., -0.1588, -0.0903, -0.3353],
        [-1.4251, -0.7188,  0.6457,  ...,  1.3513, -1.5572,  0.4356],
        [ 0.3276,  0.0985, -0.3828,  ..., -0.3682,  0.1669, -0.1790]])

In [14]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3598, -0.2233, -0.4987,  ..., -0.1588, -0.0903, -0.3353],
        [-1.4251, -0.7188,  0.6457,  ...,  1.3513, -1.5572,  0.4356],
        [ 0.3276,  0.0985, -0.3828,  ..., -0.3682,  0.1669, -0.1790]])


In [15]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [16]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [17]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [18]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text

        ########## reversing the tokens for maiking the model train in reverse order of words(tokens) ############
        text = torch.flip(text, [0])
        ##############################################

        predictions = model(text, text_lengths).squeeze(1)
        # print(predictions.shape)
        # print(batch.label.shape)
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [19]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [20]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [21]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        print('saving better model')
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

 99%|█████████▉| 397794/400000 [00:30<00:00, 20293.61it/s]

saving better model
Epoch: 01 | Epoch Time: 0m 16s
	Train Loss: 0.687 | Train Acc: 54.57%
	 Val. Loss: 0.693 |  Val. Acc: 50.90%
saving better model
Epoch: 02 | Epoch Time: 0m 16s
	Train Loss: 0.646 | Train Acc: 61.10%
	 Val. Loss: 0.670 |  Val. Acc: 58.40%
saving better model
Epoch: 03 | Epoch Time: 0m 16s
	Train Loss: 0.392 | Train Acc: 83.24%
	 Val. Loss: 0.312 |  Val. Acc: 86.01%
saving better model
Epoch: 04 | Epoch Time: 0m 16s
	Train Loss: 0.236 | Train Acc: 90.95%
	 Val. Loss: 0.293 |  Val. Acc: 88.07%
Epoch: 05 | Epoch Time: 0m 16s
	Train Loss: 0.176 | Train Acc: 93.60%
	 Val. Loss: 0.299 |  Val. Acc: 88.36%
Epoch: 06 | Epoch Time: 0m 16s
	Train Loss: 0.127 | Train Acc: 95.57%
	 Val. Loss: 0.357 |  Val. Acc: 88.51%
Epoch: 07 | Epoch Time: 0m 16s
	Train Loss: 0.094 | Train Acc: 96.92%
	 Val. Loss: 0.343 |  Val. Acc: 88.07%
Epoch: 08 | Epoch Time: 0m 16s
	Train Loss: 0.072 | Train Acc: 97.71%
	 Val. Loss: 0.401 |  Val. Acc: 87.99%
Epoch: 09 | Epoch Time: 0m 16s
	Train Loss: 0.05

In [22]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.315 | Test Acc: 87.25%


In [23]:
# import spacy
# nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [24]:
predict_sentiment(model, "This film is good")

0.9578144550323486

In [25]:
predict_sentiment(model, "This film is great")

0.9846293926239014

In [26]:
predict_sentiment(model, "This film is pathetic")

0.43270018696784973